In [2]:
import warnings
warnings.filterwarnings('ignore') 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from PIL import Image
import torchvision
from torchvision import transforms, models, datasets
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn.functional as F
from torchsummary import summary
import colorama
from colorama import Fore, Style

from torchsummary import summary

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(f'DEVICE is : {DEVICE}')

DEVICE is : cuda


In [3]:
train_dir = './train/'
valid_dir = './valid/'

classes = os.listdir(train_dir)
len(classes)

38

In [4]:
def image_count(data_dir):
        print(f'\nTrain Counting Images in Category set :')
        total_img = 0   
        # Iterate over each category (subfolder) in the split
        
        for sub_path in sorted(os.listdir(data_dir)):
                sub_cat = os.path.join(data_dir,sub_path)
                # Count the number of files, assuming they are all images
                num_images = len([name for name in os.listdir(sub_cat) if os.path.isfile(
                        os.path.join(sub_cat, name))])
                print(f"    {sub_path}: {num_images} images")
                total_img += num_images

        print(f'\n\nTotal train Images : {total_img}')

image_count(train_dir)


Train Counting Images in Category set :
    Apple___Apple_scab: 2016 images
    Apple___Black_rot: 1987 images
    Apple___Cedar_apple_rust: 1760 images
    Apple___healthy: 2008 images
    Blueberry___healthy: 1816 images
    Cherry_(including_sour)___Powdery_mildew: 1683 images
    Cherry_(including_sour)___healthy: 1826 images
    Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 1642 images
    Corn_(maize)___Common_rust_: 1907 images
    Corn_(maize)___Northern_Leaf_Blight: 1908 images
    Corn_(maize)___healthy: 1859 images
    Grape___Black_rot: 1888 images
    Grape___Esca_(Black_Measles): 1920 images
    Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 1722 images
    Grape___healthy: 1692 images
    Orange___Haunglongbing_(Citrus_greening): 2010 images
    Peach___Bacterial_spot: 1838 images
    Peach___healthy: 1728 images
    Pepper,_bell___Bacterial_spot: 1913 images
    Pepper,_bell___healthy: 1988 images
    Potato___Early_blight: 1939 images
    Potato___Late_blight: 1939

In [5]:
def image_count(data_dir):
        print(f'\nCounting Images in Category set :')
        total_img = 0
        # Iterate over each category (subfolder) in the split
        
        for sub_path in sorted(os.listdir(data_dir)):
            sub_cat = os.path.join(data_dir,sub_path)
            # Count the number of files, assuming they are all images
            num_images = len([name for name in os.listdir(sub_cat) if os.path.isfile(
                    os.path.join(sub_cat, name))])
            print(f"    {sub_path}: {num_images} images")
            total_img += num_images

        print(f'\n\nTotal Valid Images : {total_img}')
image_count(valid_dir)


Counting Images in Category set :
    Apple___Apple_scab: 504 images
    Apple___Black_rot: 497 images
    Apple___Cedar_apple_rust: 440 images
    Apple___healthy: 502 images
    Blueberry___healthy: 454 images
    Cherry_(including_sour)___Powdery_mildew: 421 images
    Cherry_(including_sour)___healthy: 456 images
    Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot: 410 images
    Corn_(maize)___Common_rust_: 477 images
    Corn_(maize)___Northern_Leaf_Blight: 477 images
    Corn_(maize)___healthy: 465 images
    Grape___Black_rot: 472 images
    Grape___Esca_(Black_Measles): 480 images
    Grape___Leaf_blight_(Isariopsis_Leaf_Spot): 430 images
    Grape___healthy: 423 images
    Orange___Haunglongbing_(Citrus_greening): 503 images
    Peach___Bacterial_spot: 459 images
    Peach___healthy: 432 images
    Pepper,_bell___Bacterial_spot: 478 images
    Pepper,_bell___healthy: 497 images
    Potato___Early_blight: 485 images
    Potato___Late_blight: 485 images
    Potato___healthy

In [6]:
train_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])

test_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])

val_transform = transforms.Compose([
    transforms.Resize((128,128)),
    transforms.ToTensor(),  # Convert images to PyTorch tensors
    transforms.Normalize(mean=[0.5,0.5,0.5], std=[0.5,0.5,0.5])  # Normalize the data to [-1, 1]
])

In [7]:
train = datasets.ImageFolder(root=train_dir, transform=train_transform)
valid = datasets.ImageFolder(root=valid_dir, transform=val_transform)

In [8]:
train

Dataset ImageFolder
    Number of datapoints: 70295
    Root location: ./train/
    StandardTransform
Transform: Compose(
               Resize(size=(128, 128), interpolation=bilinear, max_size=None, antialias=True)
               RandomHorizontalFlip(p=0.5)
               RandomRotation(degrees=[-15.0, 15.0], interpolation=nearest, expand=False, fill=0)
               ToTensor()
               Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
           )

In [9]:
train[0]

(tensor([[[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]],
 
         [[-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          ...,
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.],
          [-1., -1., -1.,  ..., -1., -1., -1.]]]),
 0)

In [10]:
img, label = train[0]
print(img.shape, label)

torch.Size([3, 128, 128]) 0


In [11]:
# def show_image(image, label):
#     print("Label :" + train.classes[label] + "(" + str(label) + ")")
#     plt.imshow(image.permute(1, 2, 0))
    
    
# image_list = [0, 3000, 5000, 8000, 12000, 15000, 60000, 70000]
    
# chs = 0
# for img in image_list:
#     chs += 1
#     plt.subplot(2,4,chs)
#     print(Fore.GREEN)
#     plt.tight_layout()
#     plt.xlabel(img,fontsize=10)
#     plt.title(train[img][1])
#     show_image(*train[img])

In [12]:
# for moving data to device (CPU or GPU)
def to_device(data, device):
    """Move tensor(s) to chosen device"""
    if isinstance(data, (list,tuple)):
        return [to_device(x, device) for x in data]
    return data.to(device, non_blocking=True)

In [13]:
# for loading in the device (GPU if available else CPU)
class DeviceDataLoader():
    """Wrap a dataloader to move data to a device"""
    def __init__(self, dataloader, device):
        self.dataloader = dataloader
        self.device = device
        
    def __iter__(self):
        """Yield a batch of data after moving it to device"""
        for b in self.dataloader:
            yield to_device(b, self.device)
        
    def __len__(self):
        """Number of batches"""
        return len(self.dataloader)

In [14]:
batch_size = 32
# DataLoaders for training and validation
train_dataloader = DataLoader(train, batch_size, shuffle=True, num_workers=2, pin_memory=True)
valid_dataloader = DataLoader(valid, batch_size, num_workers=2, pin_memory=True)

In [15]:
# Moving data into GPU, WrappedDataLoader
train_dataloader = DeviceDataLoader(train_dataloader, DEVICE)
valid_dataloader = DeviceDataLoader(valid_dataloader, DEVICE)

In [16]:
# for calculating the accuracy
def accuracy(outputs, labels):
    _, preds = torch.max(outputs, dim=1)
    return torch.tensor(torch.sum(preds == labels).item() / len(preds))

In [17]:
class ImageClassificationBase(nn.Module):
    
    def training_step(self, batch):
        images, labels = batch 
        #images, labels = images.to(DEVICE), labels.to(DEVICE) # move to GPU
        out = self(images)                  # Generate predictions
        loss = F.cross_entropy(out, labels) # Calculate loss
        return loss
    
    def validation_step(self, batch):
        images, labels = batch 
        #images, labels = images.to(DEVICE), labels.to(DEVICE) # move to GPU
        out = self(images)                    # Generate predictions
        loss = F.cross_entropy(out, labels)   # Calculate loss
        acc = accuracy(out, labels)           # Calculate accuracy
        return {'val_loss': loss.detach(), 'val_acc': acc}
        
    def validation_epoch_end(self, outputs):
        batch_losses = [x['val_loss'] for x in outputs]
        epoch_loss = torch.stack(batch_losses).mean()   # Combine losses
        batch_accs = [x['val_acc'] for x in outputs]
        epoch_acc = torch.stack(batch_accs).mean()      # Combine accuracies
        return {'val_loss': epoch_loss.item(), 'val_acc': epoch_acc.item()}
    
    def epoch_end(self, epoch, result):
        print("Epoch [{}], train_loss: {:.4f}, val_loss: {:.4f}, val_acc: {:.4f}".format(
            epoch, result['train_loss'], result['val_loss'], result['val_acc']))

In [18]:
# convolution block with BatchNormalization
def ConvBlock(in_channels, out_channels, pool=False):
    layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
             nn.BatchNorm2d(out_channels),
             nn.ReLU(inplace=True)]
    if pool:
        layers.append(nn.MaxPool2d(4))
    return nn.Sequential(*layers)

In [19]:
# resnet architecture 
class CNN_NeuralNet(ImageClassificationBase):
    def __init__(self, in_channels, num_diseases):
        super().__init__()
        
        self.conv1 = ConvBlock(in_channels, 64)
        self.conv2 = ConvBlock(64, 128, pool=True) 
        self.res1 = nn.Sequential(ConvBlock(128, 128), ConvBlock(128, 128))
        
        self.conv3 = ConvBlock(128, 256, pool=True) 
        self.conv4 = ConvBlock(256, 512, pool=True)
        #self.conv5 = ConvBlock(256, 256, pool=True)
        #self.conv6 = ConvBlock(256, 512, pool=True)
        #self.conv7 = ConvBlock(512, 512, pool=True)
        
        self.res2 = nn.Sequential(ConvBlock(512, 512), ConvBlock(512, 512))

        # self.classifier = nn.Sequential(nn.MaxPool2d(4),
        #                                nn.Flatten(),
        #                                nn.Linear(512, num_diseases))
        
        self.classifier = nn.Sequential(
                nn.AdaptiveAvgPool2d((1, 1)),  # Safe replacement
                nn.Flatten(),
                nn.Linear(512, num_diseases)
        )
        
    def forward(self, x): # x is the loaded batch
        out = self.conv1(x)
        out = self.conv2(out)
        out = self.res1(out) + out
        out = self.conv3(out)
        out = self.conv4(out)
        #out = self.conv5(out)
        #out = self.conv6(out)
        #out = self.conv7(out)
        out = self.res2(out) + out
        out = self.classifier(out)
        
        return out        

In [20]:
# defining the model and moving it to the GPU
# 3 is number of channels RGB, len(train.classes()) is number of diseases.
model = to_device(CNN_NeuralNet(3, len(train.classes)), DEVICE) 
#model = model.to(DEVICE)
model

CNN_NeuralNet(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inpl

In [21]:
# for training
@torch.no_grad()
def evaluate(model, val_loader):
    model.eval()
    outputs = [model.validation_step(batch) for batch in val_loader]
    return model.validation_epoch_end(outputs)

In [22]:
def get_lr(optimizer):
    if optimizer is None:
        raise ValueError("Optimizer is not defined.")
    if not optimizer.param_groups:
        raise ValueError("Optimizer has no param groups.")
    else:
        for param_group in optimizer.param_groups:
            return param_group['lr']

In [23]:
#%%time
# history = [evaluate(model, valid_dataloader)]
# history

### Hyperparameters Function:

Now it's time to create a function that get epochs, learning rate, train and validation loader and optim function..

Clear GPU memory after PyTorch model training without restarting kernel with torch.cuda.empty_cache()

In [ ]:
def fit_OneCycle(epochs, max_lr, model, train_loader, val_loader, weight_decay=0,
                grad_clip=None, opt_func=torch.optim.SGD):
    torch.cuda.empty_cache()
    history = []  #For collecting the results
    
    optimizer = opt_func(model.parameters(), max_lr, weight_decay=weight_decay)
    # scheduler for one cycle learniing rate
    #Sets the learning rate of each parameter group according to the 1cycle learning rate policy. 
    #The 1cycle policy anneals the learning rate from an initial learning rate to some 
    #maximum learning rate and then from that maximum learning rate to some minimum learning rate
    #much lower than the initial learning rate. 
    sched = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr,
                                                epochs=epochs, steps_per_epoch=len(train_loader))
    
    
    for epoch in range(epochs):
        # Training
        model.train()
        train_losses = []
        lrs = []
        for batch in train_loader:
            loss = model.training_step(batch)
            train_losses.append(loss)
            loss.backward()
            
            # gradient clipping
            #Clip the gradients of an iterable of parameters at specified value.
            #All from pytorch documantation.
            if grad_clip: 
                nn.utils.clip_grad_value_(model.parameters(), grad_clip)
                
            optimizer.step()
            optimizer.zero_grad()
            
            # recording and updating learning rates
            lrs.append(get_lr(optimizer))
            sched.step()
             # validation
        
        result = evaluate(model, val_loader)
        result['train_loss'] = torch.stack(train_losses).mean().item()
        result['lrs'] = lrs
        model.epoch_end(epoch, result)
        history.append(result)
        
    torch.save(model.state_dict(), 'C_Model.pth')
    return history

: 

### Training Model:

Evaluate function added to history of model.

Then we can define our hyperparameters like number of epochs, learning rate and ... .

Now we can update history with fit_OneCycle function (Adding two function together). Attention to history = [] in the second function. Now we have model evaluation.

In [ ]:
num_epoch = 5
lr_rate = 0.001
grad_clip = 0.15
weight_decay = 1e-4
optimizer = torch.optim.Adam 

history = fit_OneCycle(num_epoch, lr_rate, model, train_dataloader, valid_dataloader, 
                             grad_clip=grad_clip, 
                             weight_decay=weight_decay, 
                             opt_func=optimizer)
